<a href="https://colab.research.google.com/github/HelloShagun/useful_libs/blob/main/examples/gee_sampler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🛰️ GEE Interactive Sampler
**Draw an ROI → Load a layer → Sample points → Download**

---

In [1]:
# ── CELL 1: Install dependencies ───────────────────────────────────────────
!pip install -q geemap earthengine-api leafmap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 667.8/667.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.4/20.4 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.5/208.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.0 MB/s eta 0:00:00


In [2]:
# ── CELL 2: Authenticate & initialise Earth Engine ─────────────────────────
import ee
import geemap
import json, io, pandas as pd
from IPython.display import display, clear_output
from google.colab import files

try:
    ee.Initialize()
    print('✅ Earth Engine initialised.')
except Exception:
    ee.Authenticate()
    ee.Initialize(project="ee-shagungargc2020")
    print('✅ Earth Engine authenticated and initialised.')

✅ Earth Engine authenticated and initialised.


In [14]:
# ── CELL 3: Map — draw your ROI here ───────────────────────────────────────
# Use the rectangle or polygon tool, then run Cell 4.

Map = geemap.Map()
Map.layout.height = '500px'
Map.add_basemap('HYBRID')
display(Map)
print('👆 Draw a rectangle or polygon on the map, then run Cell 4.')

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

👆 Draw a rectangle or polygon on the map, then run Cell 4.


In [16]:
# ── CELL 4: Confirm ROI ────────────────────────────────────────────────────
# Run AFTER drawing on the map above.

drawn = Map.draw_features

if drawn is None:
    print('❌ Nothing drawn yet. Draw a shape on the map first.')
    ROI = None
else:
    ROI = ee.FeatureCollection(drawn).geometry().dissolve(maxError=1)
    coords = ROI.bounds().getInfo()['coordinates'][0]
    lon_min = min(c[0] for c in coords)
    lon_max = max(c[0] for c in coords)
    lat_min = min(c[1] for c in coords)
    lat_max = max(c[1] for c in coords)
    print(f'✅ ROI confirmed!')
    print(f'   Lon [{lon_min:.4f}, {lon_max:.4f}] | Lat [{lat_min:.4f}, {lat_max:.4f}]')
    Map.addLayer(
        ee.Image().paint(ee.FeatureCollection([ee.Feature(ROI)]), 0, 3),
        {'palette': ['orange']}, 'ROI'
    )
    Map.centerObject(ROI)

✅ ROI confirmed!
   Lon [77.5031, 78.2364] | Lat [13.0119, 13.3328]


In [17]:
# ── CELL 5: Load Dynamic World ─────────────────────────────────────────────
#
# Dynamic World bands (all are probabilities 0–1):
#   water, trees, grass, flooded_vegetation, crops,
#   shrub_and_scrub, built, bare, snow_and_ice
#
# 'label' band = index 0-8 of the highest-probability class
#   0=water  1=trees  2=grass  3=flooded_veg  4=crops
#   5=shrub  6=built  7=bare   8=snow_and_ice

# ── Edit these ────────────────────────────────────────────────────────────
START_DATE  = '2022-01-01'   # start of date range
END_DATE    = '2022-12-31'   # end of date range

# Choose ONE of the two sampling modes:
#
# MODE = 'probability'
#   Loads the 'water' probability band (float 0–1).
#   Samples ALL pixels, recording the water probability at each point.
#   Use PROBABILITY_THRESHOLD below to filter in Cell 6.
#
# MODE = 'label'
#   Loads the 'label' band (integer class 0–8).
#   Samples ONLY pixels where label == LABEL_CLASS_VALUE.
#   Good for crisp "this pixel IS water" samples.

MODE                = 'probability'   # 'probability' | 'label'
PROBABILITY_BAND    = 'water'         # which class probability to use
PROBABILITY_THRESHOLD = 0.5          # only used in MODE='probability' — min water prob to keep
LABEL_CLASS_VALUE   = 0              # only used in MODE='label' — 0=water

# ── Build the composite ───────────────────────────────────────────────────
LAYER = None

try:
    dw = (
        ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1')
        .filterDate(START_DATE, END_DATE)
        .filterBounds(ROI)
    )

    n_images = dw.size().getInfo()
    print(f'   Found {n_images} DW images in your ROI for {START_DATE} → {END_DATE}')

    if n_images == 0:
        print('❌ No images found. Try a wider date range or different ROI.')
    else:
        # Mean composite over the date range
        composite = dw.mean()

        if MODE == 'probability':
            LAYER = composite.select(PROBABILITY_BAND)
            vis = {
                'min': 0, 'max': 1,
                'palette': ['#f7fbff','#c6dbef','#6baed6','#2171b5','#08306b']
            }
            Map.addLayer(LAYER, vis, f'DW {PROBABILITY_BAND} probability ({START_DATE[:4]})')
            print(f'✅ Loaded: Dynamic World | Band: "{PROBABILITY_BAND}" probability')
            print(f'   Composite of {n_images} images | Mean mosaic')
            print(f'   Sampling will keep pixels with {PROBABILITY_BAND} ≥ {PROBABILITY_THRESHOLD}')

        else:  # label mode
            # For label mode use the most-common (mode) label, not mean
            label_composite = dw.select('label').mode()
            LAYER = label_composite
            DW_PALETTE = [
                '#419bdf','#397d49','#88b053','#7a87c6','#e49635',
                '#dfc35a','#c4281b','#a59b8f','#b39fe1'
            ]
            Map.addLayer(LAYER, {'min': 0, 'max': 8, 'palette': DW_PALETTE},
                         f'DW label mode ({START_DATE[:4]})')
            print(f'✅ Loaded: Dynamic World | Band: "label" (mode composite)')
            print(f'   Sampling will keep pixels where label == {LABEL_CLASS_VALUE} (water=0)')

        # Also show the full label layer for reference
        if MODE == 'probability':
            DW_PALETTE = [
                '#419bdf','#397d49','#88b053','#7a87c6','#e49635',
                '#dfc35a','#c4281b','#a59b8f','#b39fe1'
            ]
            Map.addLayer(
                dw.select('label').mode(),
                {'min': 0, 'max': 8, 'palette': DW_PALETTE},
                'DW label (reference)', False  # hidden by default, toggle in layer panel
            )

except Exception as e:
    print(f'❌ Error: {e}')
    LAYER = None

   Found 101 DW images in your ROI for 2022-01-01 → 2022-12-31
✅ Loaded: Dynamic World | Band: "water" probability
   Composite of 101 images | Mean mosaic
   Sampling will keep pixels with water ≥ 0.5


In [18]:
# ── CELL 6: Sampling ───────────────────────────────────────────────────────

# ── Edit these ────────────────────────────────────────────────────────────
N_POINTS      = 200      # number of sample points to collect
SAMPLING_TYPE = 'stratified' # 'random' | 'stratified' | 'grid'
SCALE         = 10       # pixel scale in metres (DW is 10 m native)
SEED          = 42

# ── Sampling type guide ───────────────────────────────────────────────────
# 'random'     → N random points in ROI, records band value at each point.
#                Results filtered by PROBABILITY_THRESHOLD (probability mode)
#                or LABEL_CLASS_VALUE (label mode).
#                RECOMMENDED for most use cases.
#
# 'stratified' → Only valid in label mode. Samples exactly from class pixels.
#
# 'grid'       → Every pixel in the ROI at SCALE. Can be large — use SCALE ≥ 100
#                for big ROIs.

# ── Run ───────────────────────────────────────────────────────────────────
SAMPLES = None

if ROI is None:
    print('❌ ROI not set — run Cell 4 first.')
elif LAYER is None:
    print('❌ Layer not loaded — run Cell 5 first.')
else:
    try:
        print(f'⏳ Running {SAMPLING_TYPE} sampling...')

        if SAMPLING_TYPE == 'random':
            # Sample N random points, get band value, then filter
            pts = ee.FeatureCollection.randomPoints(
                region=ROI, points=N_POINTS * 3, seed=SEED  # oversample then filter
            )
            sampled = LAYER.sampleRegions(
                collection=pts, scale=SCALE, geometries=True
            )
            band_name = LAYER.bandNames().get(0).getInfo()

            if MODE == 'probability':
                sampled = sampled.filter(
                    ee.Filter.gte(band_name, PROBABILITY_THRESHOLD)
                )
            else:
                sampled = sampled.filter(
                    ee.Filter.eq(band_name, LABEL_CLASS_VALUE)
                )
            # Limit to requested N
            SAMPLES = sampled.limit(N_POINTS)

        elif SAMPLING_TYPE == 'stratified':
            if MODE == 'probability':
                print('⚠️  stratified works best with label mode. Switching to threshold mask...')
                mask = LAYER.gte(PROBABILITY_THRESHOLD)
                # Convert to integer for stratifiedSample
                int_layer = mask.rename('water_mask').toInt()
                SAMPLES = int_layer.stratifiedSample(
                    numPoints=N_POINTS,
                    classBand='water_mask',
                    classValues=[1],
                    classPoints=[N_POINTS],
                    region=ROI,
                    scale=SCALE,
                    seed=SEED,
                    geometries=True
                )
            else:
                mask = LAYER.eq(LABEL_CLASS_VALUE)
                masked = LAYER.updateMask(mask).toInt()
                SAMPLES = masked.stratifiedSample(
                    numPoints=N_POINTS,
                    classBand='label',
                    classValues=[LABEL_CLASS_VALUE],
                    classPoints=[N_POINTS],
                    region=ROI,
                    scale=SCALE,
                    seed=SEED,
                    geometries=True
                )

        else:  # grid
            SAMPLES = LAYER.sample(
                region=ROI, scale=SCALE, geometries=True
            )

        # Count & display
        count = SAMPLES.size().getInfo()
        print(f'✅ Collected {count} sample points.')
        if count == 0:
            print('   Tip: lower PROBABILITY_THRESHOLD, widen date range, or enlarge ROI.')
        else:
            try:
                Map.remove_layer('Samples')
            except Exception:
                pass
            Map.addLayer(SAMPLES, {'color': 'red'}, 'Samples')

            # Preview table
            preview = SAMPLES.limit(8).getInfo()
            rows = []
            for f in preview['features']:
                row = f.get('properties', {})
                coords = f.get('geometry', {}).get('coordinates', [])
                if coords:
                    row['longitude'] = round(coords[0], 6)
                    row['latitude']  = round(coords[1], 6)
                rows.append(row)
            if rows:
                print('\n📋 Preview (first 8 rows):')
                display(pd.DataFrame(rows))

    except Exception as e:
        print(f'❌ Sampling error: {e}')
        SAMPLES = None

⏳ Running stratified sampling...
⚠️  stratified works best with label mode. Switching to threshold mask...
✅ Collected 400 sample points.

📋 Preview (first 8 rows):


,water_mask,longitude,latitude
0,0,78.100654,13.280199
1,0,77.853976,13.114370
2,0,78.057445,13.115807
3,0,77.795855,13.318288
4,0,78.077657,13.109429
5,0,77.658772,13.062447
6,0,78.104337,13.252261
7,0,78.235670,13.189918


In [19]:
# ── CELL 6b: Interactive map visualisation ─────────────────────────────────

if SAMPLES is None:
    print('❌ No samples. Run Cell 6 first.')
else:
    data = SAMPLES.getInfo()
    band_name = list(data['features'][0]['properties'].keys())[0]

    water_pts    = []
    non_water_pts = []

    for f in data['features']:
        coords = f.get('geometry', {}).get('coordinates', [])
        props  = f.get('properties', {})
        val    = props.get(band_name)
        if coords:
            if val == 1:
                water_pts.append(coords)
            else:
                non_water_pts.append(coords)

    # Add to existing map
    water_fc = ee.FeatureCollection([
        ee.Feature(ee.Geometry.Point(c)) for c in water_pts
    ])
    non_water_fc = ee.FeatureCollection([
        ee.Feature(ee.Geometry.Point(c)) for c in non_water_pts
    ])

    try:
        Map.remove_layer('Water samples')
        Map.remove_layer('Non-water samples')
    except Exception:
        pass

    Map.addLayer(water_fc,     {'color': '#419bdf'}, f'Water samples (n={len(water_pts)})')
    Map.addLayer(non_water_fc, {'color': '#e49635'}, f'Non-water samples (n={len(non_water_pts)})')

    print(f'✅ Water: {len(water_pts)} pts (blue)  |  Non-water: {len(non_water_pts)} pts (orange)')
    print('👆 Toggle layers in the map panel above (Cell 3)')
    display(Map)

✅ Water: 200 pts (blue)  |  Non-water: 200 pts (orange)
👆 Toggle layers in the map panel above (Cell 3)


Map(bottom=242982.0, center=[13.172368761849274, 77.8697185], controls=(WidgetControl(options=['position', 'tr…

In [20]:
# ── Diagnostic: inspect raw sample structure ───────────────────────────────
sample_check = SAMPLES.limit(3).getInfo()

for i, f in enumerate(sample_check['features']):
    print(f'--- Feature {i} ---')
    print('geometry :', f.get('geometry'))
    print('properties:', f.get('properties'))

--- Feature 0 ---
geometry : {'geodesic': False, 'type': 'Point', 'coordinates': [78.10065369545634, 13.28019908701674]}
properties: {'water_mask': 0}
--- Feature 1 ---
geometry : {'geodesic': False, 'type': 'Point', 'coordinates': [77.85397631843712, 13.114370085568277]}
properties: {'water_mask': 0}
--- Feature 2 ---
geometry : {'geodesic': False, 'type': 'Point', 'coordinates': [78.0574447302902, 13.115807390022868]}
properties: {'water_mask': 0}


In [21]:
# ── CELL 7: Download ───────────────────────────────────────────────────────

DOWNLOAD_FORMAT = 'CSV'              # 'CSV' | 'GeoJSON' | 'SHP'
OUTPUT_FILENAME = 'dw_water_samples'
DRIVE_FOLDER    = 'GEE_exports'      # only used for SHP

# ─────────────────────────────────────────────────────────────────────────

if SAMPLES is None:
    print('❌ No samples. Run Cell 6 first.')
else:
    try:
        if DOWNLOAD_FORMAT == 'CSV':
            print('⏳ Fetching from EE...')
            data = SAMPLES.getInfo()
            rows = []
            for f in data['features']:
                row = f.get('properties', {})
                coords = f.get('geometry', {}).get('coordinates', [])
                if coords:
                    row['longitude'] = coords[0]
                    row['latitude']  = coords[1]
                rows.append(row)
            df = pd.DataFrame(rows)
            fname = f'{OUTPUT_FILENAME}.csv'
            df.to_csv(fname, index=False)
            files.download(fname)
            print(f'✅ Downloaded: {fname}  ({len(df)} rows)')
            display(df.head())

        elif DOWNLOAD_FORMAT == 'GeoJSON':
            print('⏳ Fetching from EE...')
            data = SAMPLES.getInfo()
            fname = f'{OUTPUT_FILENAME}.geojson'
            with open(fname, 'w') as fh:
                json.dump(data, fh, indent=2)
            files.download(fname)
            print(f'✅ Downloaded: {fname}')

        elif DOWNLOAD_FORMAT == 'SHP':
            task = ee.batch.Export.table.toDrive(
                collection=SAMPLES,
                description=OUTPUT_FILENAME,
                folder=DRIVE_FOLDER,
                fileNamePrefix=OUTPUT_FILENAME,
                fileFormat='SHP'
            )
            task.start()
            print(f'✅ Export task started → Drive/{DRIVE_FOLDER}/{OUTPUT_FILENAME}.shp')
            print(f'   Check the EE Tasks panel for progress.')

    except Exception as e:
        print(f'❌ Download error: {e}')

⏳ Fetching from EE...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Downloaded: dw_water_samples.csv  (400 rows)


,water_mask,longitude,latitude
0,0,78.100654,13.280199
1,0,77.853976,13.114370
2,0,78.057445,13.115807
3,0,77.795855,13.318288
4,0,78.077657,13.109429


---
## 📝 Quick Reference

### Dynamic World bands
| Band | Type | Range | Notes |
|------|------|-------|-------|
| `water` | probability | 0–1 | use with `MODE='probability'` |
| `trees`, `grass`, `crops`, ... | probability | 0–1 | same pattern |
| `label` | integer | 0–8 | 0=water, 1=trees, 2=grass … use with `MODE='label'` |

### Which mode to use?
| Goal | Recommended settings |
|------|---------------------|
| Sample confident water pixels | `MODE='probability'`, `PROBABILITY_THRESHOLD=0.7`, `SAMPLING_TYPE='random'` |
| Sample all DW-labelled water | `MODE='label'`, `LABEL_CLASS_VALUE=0`, `SAMPLING_TYPE='stratified'` |
| Record water probability everywhere | `MODE='probability'`, `PROBABILITY_THRESHOLD=0.0`, `SAMPLING_TYPE='random'` |

### Common fixes
- **0 samples returned** → lower `PROBABILITY_THRESHOLD`, widen date range, or draw a larger ROI
- **Too slow** → increase `SCALE` (e.g. 30 or 100 m) or reduce `N_POINTS`
- **Patchy coverage** → DW only covers Sentinel-2 scenes with cloud < 35%; try a longer date range